In [44]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import numpy as np
import re
import json

In [2]:
url = "https://www.camara.cl/diputados/diputados.aspx#mostrarDiputados"
response = requests.get(url)

# Verificamos que la respuesta sea exitosa (200 = OK)
if response.status_code == 200:
    print("Página descargada con éxito")
else:
    print("Error al acceder:", response.status_code)

Página descargada con éxito


In [3]:
soup = BeautifulSoup(response.text, "html.parser")

In [4]:
contenedor = soup.find("div", id="ContentPlaceHolder1_ContentPlaceHolder1_pnlDiputadosLista")

# Dentro del div, buscar todos los artículos con clase "grid-2"
articulos = contenedor.find_all("article", class_="grid-2")

In [5]:
base_url = "https://www.camara.cl/diputados/"

diputados_data = {}

# Recorrer cada artículo
for art in articulos:
    link = art.find("a")
    if link and link.has_attr("href"):
        href = link["href"]  # ej: detalle/mociones.aspx?prmID=1096
        match = re.search(r"prmID=(\d+)", href)
        if not match:
            continue

        dip_id = match.group(1)
        print("Procesando ID:", dip_id)

        # Construir URL de biografía
        bio_url = f"https://www.camara.cl/diputados/detalle/biografia.aspx?prmId={dip_id}#ficha-diputados"
        detalle_res = requests.get(bio_url)
        detalle_soup = BeautifulSoup(detalle_res.text, "html.parser")

        # Diccionario de secciones de este diputado
        bio_dict = {}

        # Buscar div de biografía
        bio_div = detalle_soup.find("div", class_="biografia")

        if bio_div:
            # Buscar todos los párrafos
            for p in bio_div.find_all("p"):
                span = p.find("span")
                if span:
                    titulo = span.get_text(strip=True)  # ej: "Estudios"
                    span.extract()  # quitar el <span> para quedarnos con solo el contenido
                    contenido = p.get_text(" ", strip=True)  # limpiar el texto
                    bio_dict[titulo] = contenido

        # Guardar en el diccionario general
        diputados_data[dip_id] = bio_dict


# Mostrar resultado en formato JSON (bonito)
print(json.dumps(diputados_data, indent=2, ensure_ascii=False))

Procesando ID: 1096
Procesando ID: 1097
Procesando ID: 1098
Procesando ID: 1009
Procesando ID: 803
Procesando ID: 1099
Procesando ID: 1100
Procesando ID: 1101
Procesando ID: 1102
Procesando ID: 1103
Procesando ID: 1104
Procesando ID: 1012
Procesando ID: 1105
Procesando ID: 1185
Procesando ID: 1106
Procesando ID: 1107
Procesando ID: 1108
Procesando ID: 1109
Procesando ID: 971
Procesando ID: 1013
Procesando ID: 1110
Procesando ID: 815
Procesando ID: 1111
Procesando ID: 1112
Procesando ID: 1113
Procesando ID: 1015
Procesando ID: 1114
Procesando ID: 1016
Procesando ID: 1116
Procesando ID: 973
Procesando ID: 1017
Procesando ID: 1117
Procesando ID: 1019
Procesando ID: 1184
Procesando ID: 1021
Procesando ID: 975
Procesando ID: 1022
Procesando ID: 1118
Procesando ID: 976
Procesando ID: 1119
Procesando ID: 1120
Procesando ID: 1121
Procesando ID: 1122
Procesando ID: 1123
Procesando ID: 1025
Procesando ID: 1125
Procesando ID: 1126
Procesando ID: 1027
Procesando ID: 1028
Procesando ID: 1030
Proces

In [8]:
df_diputados = pd.read_csv("diputados.csv")
df_diputados.head()

,dip_id,nombre,ap_pat,ap_mat,nombre_completo,fecha_nac,sexo,sexo_valor,partido_actual_id,partido_actual_nombre,partido_actual_alias,es_vigente,n_militancias_total
0,208,Víctor,Pérez,Varela,Víctor Pérez Varela,1954-10-18,Masculino,1.0,UDI,Unión Demócrata Independiente,UDI,False,4
1,485,Jorge,Pizarro,Soto,Jorge Pizarro Soto,NaN,Masculino,1.0,DC,Partido Demócrata Cristiano,DC,False,2
2,684,Sergio,Pizarro,Mackay,Sergio Pizarro Mackay,NaN,Masculino,1.0,DC,Partido Demócrata Cristiano,DC,False,1
3,696,José Alfonso,Rodríguez,Del Río,José Alfonso Rodríguez Del Río,NaN,Masculino,1.0,RN,Renovación Nacional,RN,False,1
4,951,David,Sandoval,Plaza,David Sandoval Plaza,1952-10-20,Masculino,1.0,UDI,Unión Demócrata Independiente,UDI,False,2


In [9]:
df_vigentes = df_diputados[df_diputados["es_vigente"] == True]
df_vigentes

,dip_id,nombre,ap_pat,ap_mat,nombre_completo,fecha_nac,sexo,sexo_valor,partido_actual_id,partido_actual_nombre,partido_actual_alias,es_vigente,n_militancias_total
59,872,Jaime,Mulet,Martínez,Jaime Mulet Martínez,1963-08-03,Masculino,1.0,FRVS,Federación Regionalista Verde Social,FRVS,True,5
89,917,Gastón,Von Mühlenbrock,Zamora,Gastón Von Mühlenbrock Zamora,1954-12-26,Masculino,1.0,UDI,Unión Demócrata Independiente,UDI,True,5
92,803,René,Alinco,Bustos,René Alinco Bustos,1958-06-02,Masculino,1.0,IND,Independientes,IND,True,4
94,815,Sergio,Bobadilla,Muñoz,Sergio Bobadilla Muñoz,1958-03-25,Masculino,1.0,UDI,Unión Demócrata Independiente,UDI,True,4
157,948,Gaspar,Rivas,Sánchez,Gaspar Rivas Sánchez,1978-05-17,Masculino,1.0,IND,Independientes,IND,True,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,1135,Johannes,Kaiser,Barents-Von Hohenhagen,Johannes Kaiser Barents-Von Hohenhagen,1976-01-05,Masculino,1.0,PNL,Partido Nacional Libertario,PNL,True,3
549,1144,Christian,Matheson,Villán,Christian Matheson Villán,1957-12-17,Masculino,1.0,IND,Independientes,IND,True,1
550,1149,Carla,Morales,Maldonado,Carla Morales Maldonado,1977-10-27,Femenino,0.0,RN,Renovación Nacional,RN,True,1
551,1185,Arturo,Barrios,Oteíza,Arturo Barrios Oteíza,1967-07-22,Masculino,1.0,PS,Partido Socialista,PS,True,1


In [20]:
df_bio = pd.DataFrame.from_dict(diputados_data, orient="index")

# El índice actual son los IDs ("1096", "1097", …), lo paso a columna
df_bio = df_bio.reset_index().rename(columns={"index": "id_diputado"})
df_bio

,id_diputado,Antecedentes Familiares,Actividad Política,Actividad Parlamentaria,Profesion/Actividad,Estudios,Actividad Laboral,Otras actividades a destacar,Actividades Profesionales,Actividades Academicas,Idiomas,Publicaciones,Actividades Gremiales,Actividades preferidas,Resumen Ejecutivo
0,1096,▪ Casada. Tiene una hija y un hijo.,▪ Inicia su trayectoria política en las Juvent...,▪ En noviembre de 2021 es electa diputada para...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1097,▪ Casado.,▪ Inicia su trayectoria política al incorporar...,▪ En noviembre de 2021 es electo diputado par...,▪ Administrador Público.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1098,▪ Es madre de 4 hijas.,"▪ 2019. Inició su trayectoria política en el ""...",▪ En noviembre de 2021 es electa diputada para...,▪ Estudios Técnicos Agropecuarios.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1009,"▪ Casado. Tiene dos hijos, Jorge y Elena.","▪ Concejal, Municipalidad de Santiago. Período...",▪ En noviembre de 2017 es electo diputado para...,▪ Abogado,▪ Colegio Internacional Nido de Águilas. ▪ Der...,"▪ 2003 - 2017. Emprendedor, empresario indepen...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,803,▪ Casado con María Erita Vera. Tiene 3 hijos.,NaN,▪ Diputado en el Periodo Legislativo 2006-2010...,▪ Obrero de la construcción.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,948,NaN,NaN,▪ Diputado electo Periodos Legislativos 2010-2...,▪ Abogado.,NaN,NaN,"▪ Columnista de los diarios El Observador, El ...",▪ 2006-2009. Abogado independiente en la prov...,NaN,NaN,NaN,NaN,NaN,NaN
151,1068,NaN,NaN,▪ En noviembre de 2017 resultó electa diputada...,▪ Administradora Pública.,"▪ Administración Pública, Universidad de Chile...",NaN,NaN,NaN,NaN,NaN,NaN,▪ 2016. Presidenta de la Federación de Estudi...,NaN,NaN
152,1169,NaN,NaN,▪ En noviembre de 2021 es electa diputada para...,▪ Abogada.,▪ 1975. Enseñanza Secundaria. Liceo de Hombre...,▪ 1983 - 1988. Secretaria y Gestora Jurídica. ...,"▪ 1995-1996. Secretaria de Actas Directorio ""U...",▪ 1992 - 2000. Defensora de Oficio de la Corte...,▪ 1990. Docente asistente en materia de Derech...,NaN,NaN,NaN,NaN,NaN
153,1173,NaN,NaN,▪ En noviembre de 2021 fue electa diputada par...,NaN,"▪ Estudiante de Administración Pública,",NaN,"▪ Dirigenta estudiantil, feminista.",NaN,NaN,NaN,NaN,▪ 2015. Integra la Comisión Nacional de Estud...,NaN,NaN


In [21]:
df_bio[df_bio["Estudios"].isna() == True]

,id_diputado,Antecedentes Familiares,Actividad Política,Actividad Parlamentaria,Profesion/Actividad,Estudios,Actividad Laboral,Otras actividades a destacar,Actividades Profesionales,Actividades Academicas,Idiomas,Publicaciones,Actividades Gremiales,Actividades preferidas,Resumen Ejecutivo
0,1096,▪ Casada. Tiene una hija y un hijo.,▪ Inicia su trayectoria política en las Juvent...,▪ En noviembre de 2021 es electa diputada para...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1097,▪ Casado.,▪ Inicia su trayectoria política al incorporar...,▪ En noviembre de 2021 es electo diputado par...,▪ Administrador Público.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1098,▪ Es madre de 4 hijas.,"▪ 2019. Inició su trayectoria política en el ""...",▪ En noviembre de 2021 es electa diputada para...,▪ Estudios Técnicos Agropecuarios.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,803,▪ Casado con María Erita Vera. Tiene 3 hijos.,NaN,▪ Diputado en el Periodo Legislativo 2006-2010...,▪ Obrero de la construcción.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1099,▪ Padre de una hija y un hijo.,"▪ 2008 - 2012. Concejal independiente, Municip...",▪ En noviembre de 2021 es electo diputado para...,▪ Abogado.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,1151,NaN,NaN,▪ En noviembre de 2021 es electa diputada para...,▪ Abogada. ▪ Diplomada en Derecho Administrati...,NaN,NaN,NaN,▪ Subdirectora de Derecho Público en la Asocia...,▪ Fundadora y Académica del Observatorio de De...,NaN,NaN,NaN,NaN,NaN
147,1153,NaN,NaN,▪ En noviembre de 2021 es electa diputada para...,NaN,NaN,▪ 2021. Secretaria de la Fundación Academia In...,▪ Representante de la Asamblea de Mujeres del ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,1058,NaN,NaN,▪ En noviembre de 2017 fue electa diputada por...,▪ Abogada.,NaN,NaN,▪ 2000. Recibe premio de la Asociación de Peri...,▪ 2014. Tutora del Programa de Tutorías de la ...,▪ 2014-2017. Académica ayudante de las cátedra...,NaN,NaN,NaN,NaN,NaN
150,948,NaN,NaN,▪ Diputado electo Periodos Legislativos 2010-2...,▪ Abogado.,NaN,NaN,"▪ Columnista de los diarios El Observador, El ...",▪ 2006-2009. Abogado independiente en la prov...,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_vigentes["dip_id"] = df_vigentes["dip_id"].astype(str)
df_merge = df_vigentes.merge(
    df_bio,
    left_on="dip_id",     # nombre en df_otra
    right_on="id_diputado",  # nombre en df_bio
    how="left"            # o "inner", según lo que necesites
)

df_merge

/tmp/ipykernel_7315/3249908419.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vigentes["dip_id"] = df_vigentes["dip_id"].astype(str)


,dip_id,nombre,ap_pat,ap_mat,nombre_completo,fecha_nac,sexo,sexo_valor,partido_actual_id,partido_actual_nombre,...,Estudios,Actividad Laboral,Otras actividades a destacar,Actividades Profesionales,Actividades Academicas,Idiomas,Publicaciones,Actividades Gremiales,Actividades preferidas,Resumen Ejecutivo
0,872,Jaime,Mulet,Martínez,Jaime Mulet Martínez,1963-08-03,Masculino,1.0,FRVS,Federación Regionalista Verde Social,...,"▪ Secundarios: Liceo A Nº 7, 1º y 2º de enseña...",▪ 2010 - 2018. Abogado y emprendedor. Independ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,917,Gastón,Von Mühlenbrock,Zamora,Gastón Von Mühlenbrock Zamora,1954-12-26,Masculino,1.0,UDI,Unión Demócrata Independiente,...,▪ Secundarios. Instituto de Humanidades Luis C...,▪ 2014 al 1/09/2017. Municipalidad de las Cond...,▪ Asistencia a seminarios y cursos de actuali...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,803,René,Alinco,Bustos,René Alinco Bustos,1958-06-02,Masculino,1.0,IND,Independientes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,815,Sergio,Bobadilla,Muñoz,Sergio Bobadilla Muñoz,1958-03-25,Masculino,1.0,UDI,Unión Demócrata Independiente,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,948,Gaspar,Rivas,Sánchez,Gaspar Rivas Sánchez,1978-05-17,Masculino,1.0,IND,Independientes,...,NaN,NaN,"▪ Columnista de los diarios El Observador, El ...",▪ 2006-2009. Abogado independiente en la prov...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,1135,Johannes,Kaiser,Barents-Von Hohenhagen,Johannes Kaiser Barents-Von Hohenhagen,1976-01-05,Masculino,1.0,PNL,Partido Nacional Libertario,...,NaN,▪ Se radicó en Austria y desempeñó en distinat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,1144,Christian,Matheson,Villán,Christian Matheson Villán,1957-12-17,Masculino,1.0,IND,Independientes,...,NaN,NaN,NaN,▪ 1986. Arquitecto de la Dirección General de ...,NaN,NaN,NaN,NaN,NaN,NaN
153,1149,Carla,Morales,Maldonado,Carla Morales Maldonado,1977-10-27,Femenino,0.0,RN,Renovación Nacional,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,1185,Arturo,Barrios,Oteíza,Arturo Barrios Oteíza,1967-07-22,Masculino,1.0,PS,Partido Socialista,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
df_merge.columns

Index(['dip_id', 'nombre', 'ap_pat', 'ap_mat', 'nombre_completo', 'fecha_nac',
       'sexo', 'sexo_valor', 'partido_actual_id', 'partido_actual_nombre',
       'partido_actual_alias', 'es_vigente', 'n_militancias_total',
       'id_diputado', 'Antecedentes Familiares', 'Actividad Política',
       'Actividad Parlamentaria', 'Profesion/Actividad', 'Estudios',
       'Actividad Laboral', 'Otras actividades a destacar',
       'Actividades Profesionales', 'Actividades Academicas', 'Idiomas',
       'Publicaciones', 'Actividades Gremiales', 'Actividades preferidas',
       'Resumen Ejecutivo'],
      dtype='object')

In [40]:
# Diccionario para transformar palabras en números
num_map = {
    "un": 1, "una": 1, "uno": 1,
    "dos": 2,
    "tres": 3,
    "cuatro": 4,
    "cinco": 5,
    "seis": 6,
    "siete": 7,
    "ocho": 8,
    "nueve": 9,
    "diez": 10
}

def extraer_estado_civil(texto):
    if pd.isna(texto):
        return None
    texto = texto.lower()
    for estado in ["casado", "casada", "soltero", "soltera", "divorciado", "divorciada", "viudo", "viuda", "separado", "separada"]:
        if estado in texto:
            return estado.capitalize()
    return None

def extraer_hijos(texto):
    if pd.isna(texto):
        return None
    texto = texto.lower()

    # Buscar patrón tipo "tiene 2 hijos" o "padre de 3 hijos"
    match_num = re.search(r"(?:tiene|padre de|madre de)\s+(\d+)", texto)
    if match_num:
        return int(match_num.group(1))

    # Buscar números escritos en palabras (ej: "dos hijos")
    for palabra, numero in num_map.items():
        if re.search(rf"(?:tiene|padre de|madre de).*{palabra}", texto):
            return numero

    return None

# Aplicar a tu DataFrame
df_merge["estado_civil"] = df_merge["Antecedentes Familiares"].apply(extraer_estado_civil)
df_merge["num_hijos"] = df_merge["Antecedentes Familiares"].apply(extraer_hijos)

print(df_merge[["Antecedentes Familiares", "estado_civil", "num_hijos"]].head(20))

                              Antecedentes Familiares estado_civil  num_hijos
0                            ▪ Casado. Tiene 5 hijos.       Casado        5.0
1   ▪ Casado con Ingrid Schlatter Vollmann. Tiene ...       Casado        2.0
2      ▪ Casado con María Erita Vera.  Tiene 3 hijos.       Casado        3.0
3                                                 NaN         None        NaN
4                                                 NaN         None        NaN
5                            ▪ Casado. Tiene 2 hijos.       Casado        2.0
6                            ▪ Casado. Tiene 5 hijos.       Casado        5.0
7                         ▪ Casado. Tiene tres hijos.       Casado        3.0
8                                ▪ Casado, dos hijos.       Casado        NaN
9                                          ▪ Soltera.      Soltera        NaN
10                                ▪ Padre de un hijo.         None        1.0
11                         ▪ Casado. Tiene dos hijos.       Casa

In [49]:
def _norm_text(x: str) -> str:
    x = x.replace("▪", " ").replace("\n", " ")
    x = re.sub(r"\s+", " ", x).strip(" .\t\r")
    return x

# reglas de mapeo: (regex en minúsculas) -> etiqueta estándar
MAP = [
    (r"\b(abogad[oa])\b|ciencias jurídicas|escuela de derecho|estudiante de derecho\b|licenciad[oa] en ciencias jurídicas", "Abogado/a"),
    (r"\b(ingenier[oa]\s+comercial)\b", "Ingeniero/a Comercial"),
    (r"\b(ingenier[oa]\s+agrónom[oa])\b", "Ingeniero/a Agrónomo/a"),
    (r"\b(ingenier[oa]\s+civil\s+industrial)\b", "Ingeniero/a Civil Industrial"),
    (r"\b(ingenier[oa]\s+en\s+ejecuci[oó]n\s+industrial)\b", "Ingeniero/a en Ejecución Industrial"),
    (r"\b(ingenier[oa]\s+de\s+ejecuci[oó]n\s+en\s+administraci[oó]n(\s+de\s+empresas)?)\b", "Ingeniero/a de Ejecución en Administración"),
    (r"\b(ingenier[oa]\s+en\s+ejecuci[oó]n\s+en\s+administraci[oó]n(\s+de\s+empresas)?)\b", "Ingeniero/a de Ejecución en Administración"),
    (r"\b(administrador[oa]\s+p[úu]blic[oa])\b", "Administrador/a Público/a"),
    (r"\b(administrador[oa]\s+de\s+empresas)\b|licenciad[oa]\s+en\s+administraci[oó]n\s+y\s+direcci[oó]n\s+de\s+empresas\b", "Administrador/a de Empresas"),
    (r"\b(matr[oó]n[ao])\b", "Matrona/Matron"),
    (r"\b(sociólog[oa])\b", "Sociólogo/a"),
    (r"\b(periodista)\b", "Periodista"),
    (r"\b(diseñador[oa]\s+industrial)\b", "Diseñador/a Industrial"),
    (r"\b(diseñador[oa]\s+gr[aá]fico)\b|gr[aá]fico\s+publicitario", "Diseñador/a Gráfico/a"),
    (r"\b(publicista|t[eé]cnico\s+en\s+publicidad)\b", "Publicista"),
    (r"\b(arquitect[oa])\b", "Arquitecto/a"),
    (r"\b(m[eé]dic[oa]\s+cirujan[oa])\b|\bm[eé]dic[oa]\b", "Médico/a"),
    (r"\b(cirujan[oa]\s+dentista)\b|odont[oó]log[oa]", "Cirujano/a Dentista"),
    (r"\b(b[ií]olog[oa]\s+marin[oa])\b", "Biólogo/a Marino/a"),
    (r"\b(veterinari[oa])\b", "Veterinario/a"),
    (r"\b(contador\s+auditor)\b", "Contador Auditor"),
    (r"\b(t[eé]cnic[oa]\s+en\s+administraci[oó]n\s+de\s+empresas)\b", "Técnico/a en Administración de Empresas"),
    (r"\b(t[eé]cnic[oa]\s+en\s+qu[ií]mica)\b", "Técnico/a en Química"),
    (r"\b(educador[oa]\s+de\s+p[aá]rvulos)\b", "Educador/a de Párvulos"),
    (r"\b(profesor[oa]\s+de\s+educaci[oó]n\s+b[aá]sica)\b", "Profesor/a de Educación Básica"),
    (r"\b(profesor[oa]\s+de\s+ingl[eé]s(\s+de\s+enseñanza\s+media)?)\b|\b(profesor[oa]\s+de\s+idiomas)\b", "Profesor/a de Inglés"),
    (r"\b(profesor[oa]\s+de\s+historia\s+y\s+geograf[ií]a)\b", "Profesor/a de Historia y Geografía"),
    (r"\b(asistente\s+social)\b|\b(trabajador[oa]\s+social)\b|\begresad[oa]\s+de\s+trabajo\s+social\b", "Trabajador/a Social"),
    (r"\b(administrador[oa]\s+de\s+empresas\.)\b", "Administrador/a de Empresas"),
    (r"\b(cientista\s+pol[ií]tico)\b", "Cientista Político/a"),
    (r"\b(cientista\s+criminal[ií]stic[oa])\b", "Cientista Criminalístico/a"),
    (r"\b(agente\s+comercial)\b", "Agente Comercial"),
    (r"\b(corredor\s+de\s+propiedades)\b", "Corredor/a de Propiedades"),
    (r"\b(administrador[oa]\s+p[úu]blico)\b", "Administrador/a Público/a"),
    (r"\b(licenciad[oa]\s+en\s+psicolog[ií]a)\b|abogado\s+y\s+psic[oó]logo", "Psicólogo/a"),
    (r"\b(ingenier[oa]\s+en\s+administraci[oó]n\s+log[ií]stica)\b", "Ingeniero/a en Administración Logística"),
    (r"\b(licenciad[oa]\s+en\s+ciencias\s+econ[oó]micas\s+y\s+administrativas)\b", "Lic. en Cs. Económicas y Administrativas"),
    (r"\b(estudios\s+t[eé]cnicos\s+agropecuarios)\b", "Estudios Técnicos Agropecuarios"),
    (r"\b(estudios\s+de\s+derecho)\b", "Estudios de Derecho"),
    (r"\b(estudiante\s+de\s+derecho)\b", "Estudiante de Derecho"),
    (r"\b(actor|actriz)\b", "Actor/Actriz"),
    (r"\b(docente)\b", "Docente"),
    (r"\b(m[uú]sic[oa])\b", "Músico/a"),
    (r"\b(comunicador\s+social)\b", "Comunicador/a Social"),
    (r"\b(dirigent[ea]\s+social\s+mapuche)\b", "Dirigente Social Mapuche"),
    (r"\b(activista\s+medioambiental)\b", "Activista medioambiental"),
    (r"\b(constructor\s+civil)\b", "Constructor Civil"),
    (r"\b(licenciad[oa]\s+en\s+administraci[oó]n\s+y\s+direcci[oó]n\s+de\s+empresas)\b", "Lic. en Administración y Dirección de Empresas"),
    (r"\b(master\s+en\s+derecho\s+internacional\s+de\s+los\s+derechos\s+humanos)\b", "Master en Derecho Internacional DD.HH."),
    # frases biográficas que NO son profesión → descartar
    (r"^\d{4}\b.*", ""),  # líneas que comienzan con año
    (r"\ben\s+el\s+año\b.*", ""),
    (r"\btrabaj[oó]\b.*", ""),
    (r"\bdesde\s+muy\s+joven\b.*", ""),
    (r"\buniversidad\b.*", ""),  # si queda solo línea con “Universidad …”
]

def map_item(item: str) -> str:
    s = _norm_text(item).lower()
    # separar por puntos/guiones medios si vienen pegados
    parts = re.split(r"[•|;]|(?<=\S)\.\s+| \u2022 ", s)  # divide también por punto+espacio
    out = []
    for p in parts:
        p = _norm_text(p)
        if not p:
            continue
        label = None
        for rx, lab in MAP:
            if re.search(rx, p, flags=re.IGNORECASE):
                label = lab
                if lab == "":  # regla de descarte
                    label = None
                break
        if label is None:
            # si es algo corto y sin verbo, capitaliza y conserva (p.ej., “Agricultor”)
            if re.match(r"^[a-záéíóúñü\s/]+$", p, flags=re.IGNORECASE):
                p_clean = p.strip(" .").title()
                # normalizaciones rápidas
                p_clean = p_clean.replace("Administradora", "Administrador/a").replace("Administrador", "Administrador/a") \
                                 .replace("Abogada", "Abogado/a").replace("Abogado", "Abogado/a") \
                                 .replace("Ingeniera", "Ingeniero/a").replace("Ingeniero", "Ingeniero/a") \
                                 .replace("Médica", "Médico/a").replace("Médico", "Médico/a") \
                                 .replace("Actriz", "Actor/Actriz").replace("Actor", "Actor/Actriz")
                label = p_clean
        if label:
            out.append(label)
    # dedup preservando orden
    seen = set()
    uniq = []
    for a in out:
        if a not in seen:
            seen.add(a)
            uniq.append(a)
    return " | ".join(uniq) if uniq else np.nan

def estandarizar_profesion(col: pd.Series) -> pd.Series:
    return col.map(lambda v: map_item(v) if isinstance(v, str) else np.nan)

# === Uso ===
df_merge["Profesion_std"] = estandarizar_profesion(df_merge["Profesion/Actividad"])

In [58]:
# --- PARCHE: reemplaza solo estas dos funciones ---

def parse_estudios_cell(cell) -> dict:
    if not isinstance(cell, str) or not cell.strip():
        # SIEMPRE incluir 'std'
        return {"items": [], "nivel_max": None, "principal": {}, "std": None}

    segs = split_segments(cell)
    items = []
    for seg in segs:
        seg_n = _norm_terms(seg)
        level = infer_level(seg_n)
        y1, y2 = extract_years(seg_n)
        inst = extract_institution(seg_n)
        area = extract_area(seg_n)
        grado = tidy_degree_name(seg_n, level, area)

        if level in ("Básica","Secundaria") and not inst and not area and not y1:
            continue

        items.append({
            "nivel": level,
            "grado": grado,
            "area": area,
            "institucion": inst,
            "anio_inicio": y1,
            "anio_fin": y2
        })

    def level_score(it):
        return LEVEL_ORDER.get(it["nivel"], 0)

    principal = {}
    if items:
        items_sorted = sorted(
            items,
            key=lambda d: (level_score(d), (d["anio_fin"] or d["anio_inicio"] or "0000")),
            reverse=True
        )
        principal = items_sorted[0]

    def fmt_item(it):
        y = ""
        if it["anio_inicio"] and it["anio_fin"]:
            y = f" ({it['anio_inicio']}-{it['anio_fin']})"
        elif it["anio_inicio"] and not it["anio_fin"]:
            y = f" ({it['anio_inicio']})"
        core = it["grado"] if it["grado"] else (it["area"] or it["nivel"])
        if it["institucion"]:
            core = f"{core} — {it['institucion']}"
        return f"{it['nivel']}: {core}{y}"

    std_list = [fmt_item(it) for it in items]
    std = " | ".join(std_list) if std_list else None

    return {
        "items": items,
        "nivel_max": principal.get("nivel"),
        "principal": principal,
        "std": std,  # <- clave garantizada
    }

def estandarizar_estudios(col: pd.Series):
    parsed = col.map(parse_estudios_cell)

    # robustez por si algo no fuera dict (no debería ocurrir, pero por si acaso)
    def safe_get(d, k):
        return d.get(k) if isinstance(d, dict) else None

    out        = pd.Series(parsed.map(lambda d: safe_get(d, "std")), index=col.index, name="Estudios_std")
    nivel_max  = parsed.map(lambda d: safe_get(d, "nivel_max"))
    prin       = parsed.map(lambda d: safe_get(d, "principal"))

    titulo = prin.map(lambda d: d.get("grado") if isinstance(d, dict) else None)
    area   = prin.map(lambda d: d.get("area") if isinstance(d, dict) else None)
    inst   = prin.map(lambda d: d.get("institucion") if isinstance(d, dict) else None)
    y1     = prin.map(lambda d: d.get("anio_inicio") if isinstance(d, dict) else None)
    y2     = prin.map(lambda d: d.get("anio_fin") if isinstance(d, dict) else None)

    anhos = np.where(
        (pd.Series(y1).notna()) | (pd.Series(y2).notna()),
        pd.Series(y1).fillna("") + np.where(pd.Series(y1).notna() & pd.Series(y2).notna(), "-", "") + pd.Series(y2).fillna(""),
        None
    )

    return pd.DataFrame({
        "Estudios_std": out,
        "Nivel_maximo": nivel_max,
        "Titulo_principal": titulo,
        "Area_principal": area,
        "Institucion_principal": inst,
        "Anhos_principales": anhos
    })

In [59]:
df_est = estandarizar_estudios(df_merge["Estudios"])
df_merge = pd.concat([df_merge, df_est], axis=1)

In [55]:
df_merge["Profesion_std"].value_counts()

Profesion_std
Abogado/a                                                                                                                                                              42
Ingeniero/a Comercial                                                                                                                                                  12
Administrador/a Público                                                                                                                                                 6
Médico/a                                                                                                                                                                6
Periodista                                                                                                                                                              4
Trabajador/a Social                                                                                                                     

In [31]:
df_detalle = pd.read_csv("detalle_voto.csv")
df_detalle

,Id,Descripcion,Fecha,TotalSi,TotalNo,TotalAbstencion,TotalDispensado,Quorum._value_1,Quorum.Valor,Resultado._value_1,...,Diputado.ApellidoPaterno,Diputado.ApellidoMaterno,Diputado.FechaNacimiento,Diputado.FechaDefucion,Diputado.RUT,Diputado.RUTDV,Diputado.Sexo,Diputado.Militancias,OpcionVoto._value_1,OpcionVoto.Valor
0,39821,Boletín N° 15557-05,2022-12-21,85,9,4,0,Quórum Simple,1,Aprobado,...,Acevedo,Sáez,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
1,39821,Boletín N° 15557-05,2022-12-21,85,9,4,0,Quórum Simple,1,Aprobado,...,Ahumada,Palma,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
2,39821,Boletín N° 15557-05,2022-12-21,85,9,4,0,Quórum Simple,1,Aprobado,...,Alessandri,Vergara,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
3,39821,Boletín N° 15557-05,2022-12-21,85,9,4,0,Quórum Simple,1,Aprobado,...,Alinco,Bustos,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
4,39821,Boletín N° 15557-05,2022-12-21,85,9,4,0,Quórum Simple,1,Aprobado,...,Araya,Guerrero,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765971,52568,1-Otros,2025-01-06,63,26,11,0,Quórum Simple,1,Aprobado,...,Venegas,Salazar,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0
765972,52568,1-Otros,2025-01-06,63,26,11,0,Quórum Simple,1,Aprobado,...,Von Mühlenbrock,Zamora,NaN,NaN,NaN,NaN,NaN,NaN,Abstención,2.0
765973,52568,1-Otros,2025-01-06,63,26,11,0,Quórum Simple,1,Aprobado,...,Weisse,Novoa,NaN,NaN,NaN,NaN,NaN,NaN,En Contra,0.0
765974,52568,1-Otros,2025-01-06,63,26,11,0,Quórum Simple,1,Aprobado,...,Winter,Etcheberry,NaN,NaN,NaN,NaN,NaN,NaN,Afirmativo,1.0


In [33]:
len(df_detalle["Id"].unique())

5980

In [35]:
df_detalle.columns

Index(['Id', 'Descripcion', 'Fecha', 'TotalSi', 'TotalNo', 'TotalAbstencion',
       'TotalDispensado', 'Quorum._value_1', 'Quorum.Valor',
       'Resultado._value_1', 'Resultado.Valor', 'Tipo._value_1', 'Tipo.Valor',
       'Votos.Voto', 'Votos', 'Diputado.Id', 'Diputado.Nombre',
       'Diputado.Nombre2', 'Diputado.ApellidoPaterno',
       'Diputado.ApellidoMaterno', 'Diputado.FechaNacimiento',
       'Diputado.FechaDefucion', 'Diputado.RUT', 'Diputado.RUTDV',
       'Diputado.Sexo', 'Diputado.Militancias', 'OpcionVoto._value_1',
       'OpcionVoto.Valor'],
      dtype='object')

In [63]:
df_merge[df_merge["nombre"] == "René"]

,dip_id,nombre,ap_pat,ap_mat,nombre_completo,fecha_nac,sexo,sexo_valor,partido_actual_id,partido_actual_nombre,...,estado_civil,num_hijos,profesion_std,Profesion_std,Estudios_std,Nivel_maximo,Titulo_principal,Area_principal,Institucion_principal,Anhos_principales
2,803,René,Alinco,Bustos,René Alinco Bustos,1958-06-02,Masculino,1.0,IND,Independientes,...,Casado,3.0,obrero de la construcción,Obrero De La Construcción,None,None,None,None,None,None


In [64]:
df_diputados

,dip_id,nombre,ap_pat,ap_mat,nombre_completo,fecha_nac,sexo,sexo_valor,partido_actual_id,partido_actual_nombre,partido_actual_alias,es_vigente,n_militancias_total
0,208,Víctor,Pérez,Varela,Víctor Pérez Varela,1954-10-18,Masculino,1.0,UDI,Unión Demócrata Independiente,UDI,False,4
1,485,Jorge,Pizarro,Soto,Jorge Pizarro Soto,NaN,Masculino,1.0,DC,Partido Demócrata Cristiano,DC,False,2
2,684,Sergio,Pizarro,Mackay,Sergio Pizarro Mackay,NaN,Masculino,1.0,DC,Partido Demócrata Cristiano,DC,False,1
3,696,José Alfonso,Rodríguez,Del Río,José Alfonso Rodríguez Del Río,NaN,Masculino,1.0,RN,Renovación Nacional,RN,False,1
4,951,David,Sandoval,Plaza,David Sandoval Plaza,1952-10-20,Masculino,1.0,UDI,Unión Demócrata Independiente,UDI,False,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,1135,Johannes,Kaiser,Barents-Von Hohenhagen,Johannes Kaiser Barents-Von Hohenhagen,1976-01-05,Masculino,1.0,PNL,Partido Nacional Libertario,PNL,True,3
549,1144,Christian,Matheson,Villán,Christian Matheson Villán,1957-12-17,Masculino,1.0,IND,Independientes,IND,True,1
550,1149,Carla,Morales,Maldonado,Carla Morales Maldonado,1977-10-27,Femenino,0.0,RN,Renovación Nacional,RN,True,1
551,1185,Arturo,Barrios,Oteíza,Arturo Barrios Oteíza,1967-07-22,Masculino,1.0,PS,Partido Socialista,PS,True,1
